In [3]:
#Criando o mundo
import numpy as np
import random
class WumpusWorld:
    def __init__(self):
        # O mapa é uma grade 4x4. Coordenadas (coluna, linha) de 1 a 4.
        # Definimos os elementos fixos do mapa.
        self.grid_size = 4
        self.wumpus_pos = (4, 2)  # Coluna 4, Linha 4
        self.pit_positions = [(3, 1), (2, 3), (3, 4)]
        self.gold_pos = (4, 4)
        self.start_pos = (1, 1) # Agente sempre começa em [1,1]
        self.agent_pos = self.start_pos
        self.wumpus_alive = True
        self.has_gold = False
        self.arrows = 1 # Agente possui somente uma chance de atirar.

        # Direções para movimento: Norte, Leste, Sul, Oeste
        self.directions = {
            'N': (0, 1),
            'E': (1, 0),
            'S': (0, -1),
            'W': (-1, 0)
        }
        self.current_direction_idx = 0

    def reset(self):
        """Redefine o ambiente para o estado inicial."""
        self.agent_pos = self.start_pos
        self.wumpus_alive = True
        self.has_gold = False
        self.arrows = 1
        self.current_direction_idx = 0
        return self._get_state()

    def _get_state(self):
        """
        Retorna o estado atual do agente.
        O estado pode ser uma tupla de (pos_x, pos_y, wumpus_alive, has_gold, current_direction_idx).
        Para um grid 4x4, isso seria (1-4, 1-4, 0/1, 0/1, 0-3).
        """
        return (self.agent_pos[0], self.agent_pos[1], int(self.wumpus_alive), int(self.has_gold), self.current_direction_idx)

    def is_valid_position(self, x, y):
        """Verifica se uma posição (x, y) está dentro dos limites do mapa."""
        if 1 <= x <= self.grid_size and 1 <= y <= self.grid_size:
            return True
        else:
            return False

    def is_death_trap(self, x, y):
        """Verifica se a posição (x, y) é um abismo ou contém um Wumpus vivo."""
        if (x, y) in self.pit_positions:
            return "pit" # Retorna "pit" se for um abismo
        if (x, y) == self.wumpus_pos and self.wumpus_alive:
            return "wumpus" # Retorna "wumpus" se for o Wumpus vivo
        return None

    def get_perceptions(self, agent_x, agent_y):
        """
        Retorna as percepções do agente na posição (agent_x, agent_y).
        As percepções são: cheiro (stench), brisa (breeze), brilho (glitter), choque (bump).
        Um grito (scream) é global e acontece quando o Wumpus é morto.
        """
        perceptions = []

        # Verificar brilho (Glitter)
        if (agent_x, agent_y) == self.gold_pos:
            perceptions.append("Brilho") # Agente percebe um brilho no compartimento onde o ouro está

        # Verificar cheiro (Stench)
        # Wumpus está no compartimento (wumpus_x, wumpus_y) ou em um compartimento adjacente
        wumpus_x, wumpus_y = self.wumpus_pos
        if ((agent_x == wumpus_x and abs(agent_y - wumpus_y) == 1) or
            (agent_y == wumpus_y and abs(agent_x - wumpus_x) == 1) or
            (agent_x, agent_y) == self.wumpus_pos):
            perceptions.append("Fedor") # No compartimento que contém o Wumpus e nos adjacentes, o agente percebe cheiro ruim

        # Verificar brisa (Breeze)
        # Abismos estão em pit_positions
        for pit_x, pit_y in self.pit_positions:
            if ((agent_x == pit_x and abs(agent_y - pit_y) == 1) or
                (agent_y == pit_y and abs(agent_x - pit_x) == 1)):
                perceptions.append("Brisa") # Nos compartimentos adjacentes a um abismo, o agente percebe uma brisa
                break # Uma brisa já é suficiente

        return perceptions

    def show_full_map(self):
        """
        Imprime o mapa completo do Mundo do Wumpus no console.
        'W': Wumpus
        'P': Abismo (Pit)
        'G': Ouro (Gold)
        'S': Início (Start) / Agente
        '.': Vazio
        """
        print("\n--- Mapa do Mundo do Wumpus ---")
        # Itera pelas linhas de cima para baixo (grid_size até 1)
        for y in range(self.grid_size, 0, -1):
            row_str = ""
            for x in range(1, self.grid_size + 1):
                cell_content = '.'
                if (x, y) == self.wumpus_pos:
                    cell_content = 'W'
                elif (x, y) == self.gold_pos:
                    cell_content = 'G'
                elif (x, y) in self.pit_positions:
                    cell_content = 'P'
                elif (x, y) == self.start_pos:
                    cell_content = 'S' # A posição inicial também pode ter um item, mas para este exemplo, o agente "está" ali.

                row_str += f"[{cell_content}] "
            print(row_str)
        print("-------------------------------\n")
    def step(self, action):
          """
        Executa uma ação no ambiente e retorna (novo_estado, recompensa, done).
        Ações:
        0: Mover para frente
        1: Virar à direita
        2: Virar à esquerda
        3: Pegar ouro
        4: Atirar flecha
        5: Sair da caverna (apenas em [1,1] com ouro)
        Recompensas:
        Local vazio: -1
        Sentir brisa: -2
        Sentir Fedor: -2
        Sentir brilho: 10
        Morrer pro wumpus: -100
        Atirar: 20 #Apenas ganha se atira quando sente fedor
        Matar o wumpus: 50
        Sair da caverna: 100
        Cair num buraco: -50
        """
          reward = -1 # Recompensa base por cada ação, Local vazio: -1
          done = False
          (agent_x, agent_y) = self.agent_pos
          (next_x, next_y) = (agent_x, agent_y)

          # Direção atual do agente
          current_dir_vec = list(self.directions.values())[self.current_direction_idx]

          if action == 0:  # Mover para frente
              next_x = agent_x + current_dir_vec[0]
              next_y = agent_y + current_dir_vec[1]
              if self.is_valid_position(next_x, next_y):
                  self.agent_pos = (next_x, next_y)
                  death_reason = self.is_death_trap(next_x, next_y)
                  if death_reason == "wumpus":
                      reward = -100 # Morrer pro wumpus: -100
                      done = True
                  elif death_reason == "pit":
                      reward = -50 # Cair num buraco: -50
                      done = True
              else:
                  pass
          elif action == 1:  # Virar à direita
              self.current_direction_idx = (self.current_direction_idx + 1) % 4
          elif action == 2:  # Virar à esquerda
              self.current_direction_idx = (self.current_direction_idx - 1 + 4) % 4 # Garante resultado positivo
          elif action == 3:  # Pegar ouro
              if self.agent_pos == self.gold_pos and not self.has_gold:
                  self.has_gold = True
                  # Recompensa por pegar ouro não é explícita aqui, mas é implicitamente incentivada
                  # por permitir a ação 'Sair da caverna' com alta recompensa.
              else:
                  # Ação inválida ou ouro já pego. Punição leve.
                  pass
          elif action == 4:  # Atirar flecha
              if self.arrows > 0:
                  self.arrows -= 1
                  current_perceptions = self.get_perceptions(agent_x, agent_y)
                  if "Fedor" in current_perceptions:
                    reward += 20
                  # Calcular a posição alvo da flecha (um passo à frente)
                  target_x = agent_x + current_dir_vec[0]
                  target_y = agent_y + current_dir_vec[1]

                  # Se o Wumpus estiver na célula adjacente na direção que o agente está virado:
                  if self.wumpus_alive and (target_x, target_y) == self.wumpus_pos:
                      self.wumpus_alive = False
                      reward += 50 # Matar o wumpus: 50
              else:
                  # Ação inválida: Tentar atirar sem flechas. Não adiciona penalidade extra,
                  # apenas o custo base de -1 pela ação já é suficiente para desincentivar.
                  pass
          elif action == 5:  # Sair da caverna
              if self.agent_pos == self.start_pos and self.has_gold:
                  reward = 100 # Sair da caverna: 100 (substitui a penalidade base)
                  done = True
              else:
                  pass

          return self._get_state(), reward, done
if __name__ == "__main__": #Teste de percepções
    world = WumpusWorld()

    print(f"Percepções em [1,1]: {world.get_perceptions(1, 1)}")
    print(f"Percepções em [1,2]: {world.get_perceptions(1, 2)}")
    print(f"Percepções em [1,3]: {world.get_perceptions(1, 3)}") # Ouro
    print(f"Percepções em [2,1]: {world.get_perceptions(2, 1)}")
    print(f"Percepções em [2,2]: {world.get_perceptions(2, 2)}")
    print(f"Percepções em [3,2]: {world.get_perceptions(3, 2)}") # Abismo

    print(f"É uma armadilha mortal em [3,1]? {world.is_death_trap(3, 1)}") # Abismo
    print(f"É uma armadilha mortal em [4,4]? {world.is_death_trap(4, 4)}") # Wumpus
    print(f"É uma armadilha mortal em [1,1]? {world.is_death_trap(1, 1)}") # Início

#Feito com auxilio do Gemini

Percepções em [1,1]: []
Percepções em [1,2]: []
Percepções em [1,3]: ['Brisa']
Percepções em [2,1]: ['Brisa']
Percepções em [2,2]: ['Brisa']
Percepções em [3,2]: ['Fedor', 'Brisa']
É uma armadilha mortal em [3,1]? pit
É uma armadilha mortal em [4,4]? None
É uma armadilha mortal em [1,1]? None


In [2]:
#Mundo criado
world.show_full_map()


--- Mapa do Mundo do Wumpus ---
[.] [.] [P] [G] 
[.] [P] [.] [.] 
[.] [.] [.] [W] 
[S] [.] [P] [.] 
-------------------------------



In [4]:
class QLearningAgent:
    def __init__(self, env, learning_rate=0.1, discount_factor=0.9, epsilon=1.0, epsilon_decay_rate=0.01, min_epsilon=0.01):
        self.env = env
        self.lr = learning_rate # Taxa de aprendizado (alpha)
        self.gamma = discount_factor # Fator de desconto (gamma)
        self.epsilon = epsilon # Taxa de exploração (epsilon)
        self.epsilon_decay_rate = epsilon_decay_rate
        self.min_epsilon = min_epsilon

        # Ações: Mover Frente, Virar Direita, Virar Esquerda, Pegar Ouro, Atirar Flecha, Escalar
        self.actions = [0, 1, 2, 3, 4, 5]
        self.num_actions = len(self.actions)

        self.q_table = {}

    def _get_q_value(self, state, action):
        """Retorna o valor Q para um dado estado e ação, ou 0 se não existir."""
        return self.q_table.get((state, action), 0.0)

    def choose_action(self, state):
        """Escolhe uma ação usando a política epsilon-greedy."""
        if random.uniform(0, 1) < self.epsilon:
            return random.choice(self.actions) # Exploração: escolhe uma ação aleatória
        else:
            # Exploração com base na ação com o maior valor Q
            q_values = [self._get_q_value(state, action) for action in self.actions]
            max_q = max(q_values)
            # Em caso de empate, escolha uma aleatoriamente entre as melhores
            best_actions = [self.actions[i] for i, q in enumerate(q_values) if q == max_q]
            return random.choice(best_actions)

    def learn(self, state, action, reward, next_state, done):
        """Atualiza a tabela Q com base na experiência."""
        current_q = self._get_q_value(state, action)

        if done:
            target_q = reward
        else:
            # Encontra o valor Q máximo para o próximo estado
            max_next_q = max([self._get_q_value(next_state, a) for a in self.actions])
            target_q = reward + self.gamma * max_next_q

        # Atualiza o valor Q
        self.q_table[(state, action)] = current_q + self.lr * (target_q - current_q)

    def decay_epsilon(self):
        """Diminui o valor de epsilon."""
        self.epsilon = max(self.min_epsilon, self.epsilon - self.epsilon_decay_rate)

In [11]:
#Ele não encontrou uma saida com menos de 450 episodios
#Assim, temos uma solução
num_episodes = 450
max_steps_per_episode = 200 # Limite de passos para evitar loops infinitos

# Inicializar ambiente e agente
world = WumpusWorld()
agent = QLearningAgent(world)

# Armazenar recompensas para plotagem ou análise
rewards_per_episode = []

for episode in range(num_episodes):
    state = world.reset() # Redefine o ambiente para o início de cada episódio
    total_reward = 0
    done = False
    step = 0

    while not done and step < max_steps_per_episode:
        action = agent.choose_action(state)
        next_state, reward, done = world.step(action)
        agent.learn(state, action, reward, next_state, done)

        state = next_state
        total_reward += reward
        step += 1

    rewards_per_episode.append(total_reward)
    agent.decay_epsilon()

    if (episode + 1) % 1000 == 0:
        print(f"Episódio {episode + 1}/{num_episodes}, Recompensa Total: {total_reward}, Epsilon: {agent.epsilon:.4f}")

# Exemplo de como o agente se comportaria após o treinamento (política ótima)
print("\n--- Testando o agente treinado ---")
state = world.reset()
done = False
total_reward = 0
path = [world.agent_pos]
test_steps = 0

while not done and test_steps < max_steps_per_episode * 2: # Mais passos para o teste
    # Escolhe a ação com o maior Q-value (sem exploração)
    q_values = [agent._get_q_value(state, action) for action in agent.actions]
    if not q_values: # Caso extremo onde o estado-ação não foi explorado
        action = random.choice(agent.actions)
    else:
        max_q = max(q_values)
        best_actions = [agent.actions[i] for i, q in enumerate(q_values) if q == max_q]
        action = random.choice(best_actions) # Em caso de empate

    next_state, reward, done = world.step(action)
    total_reward += reward
    state = next_state
    path.append(world.agent_pos)
    test_steps += 1

    if done:
        death_reason = world.is_death_trap(world.agent_pos[0], world.agent_pos[1])
        if world.agent_pos == world.start_pos and world.has_gold:
            print("Agente encontrou o ouro e escapou!")
        elif death_reason == "wumpus":
            print("Agente morreu para o Wumpus!")
        elif death_reason == "pit":
            print("Agente caiu em um buraco!")
        else: # Outras condições de 'done' (e.g., max_steps)
            print("Episódio de teste terminado por limite de passos ou condição desconhecida.")
        break

print(f"Recompensa Final no Teste: {total_reward}")
print(f"Caminho percorrido: {path}")
print(f"Tamanho da Q-table: {len(agent.q_table)}")

# Você pode plotar rewards_per_episode para visualizar o aprendizado
# import matplotlib.pyplot as plt
# plt.plot(rewards_per_episode)
# plt.xlabel("Episódios")
# plt.ylabel("Recompensa Total")
# plt.title("Recompensa por Episódio no Mundo do Wumpus")
# plt.show()


--- Testando o agente treinado ---
Agente encontrou o ouro e escapou!
Recompensa Final no Teste: 150
Caminho percorrido: [(1, 1), (1, 2), (1, 2), (2, 2), (3, 2), (3, 2), (4, 2), (4, 2), (4, 3), (4, 4), (4, 4), (4, 4), (4, 4), (4, 3), (4, 2), (4, 2), (3, 2), (2, 2), (1, 2), (1, 2), (1, 1), (1, 1)]
Tamanho da Q-table: 796
